In [1]:
import os
import requests
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

True

환경변수 설정

In [2]:
aoai_endpoint = os.getenv("AZURE_OPENAI_ENDPONT")
aoai_key = os.getenv("AZURE_OPENAI_KEY")
search_endpoint = os.getenv("AI_SEARCH_ENDPOINT")
search_key = os.getenv("AI_SEARCH_KEY")

질문 벡터화

In [3]:
# Azure OpenAI Client
client = AzureOpenAI(
    azure_endpoint=aoai_endpoint,  # 엔드포인트
    api_key=aoai_key,  # 키
    api_version="2024-10-21"  # API 버전
)

In [4]:
def query_embedding(question: str):
    embedding = client.embeddings.create(
        model="text-embedding-ada-002", # model = "deployment_name".
        input=question
    )
    return embedding.data[0].embedding

Search 검색 함수 정의

In [5]:
# 키워드 검색
def text_search(question, index_name, top_p = 5):
    base_url = f'{search_endpoint}/indexes/{index_name}/docs/search'
    query_params = {'api-version': '2023-11-01'}
    headers = {'Content-Type': 'application/json', 'api-key': search_key}
    payload = {
        "search": question,
        "queryType": "simple",  # simple, full, semantic
        "searchMode": "all",
        "searchFields": "title, chunk",  # 검색할 필드
        "select": "title, chunk",
        "top": top_p,
        "count": "true"
    }
    response = requests.post(base_url, params=query_params, headers=headers, json=payload)
    response_body = response.json()
    return response_body
        

In [6]:
# 벡터 검색
def vector_search(question, index_name, top_k = 5):
    base_url = f'{search_endpoint}/indexes/{index_name}/docs/search'
    query_params = {'api-version': '2023-11-01'}
    headers = {'Content-Type': 'application/json', 'api-key': search_key}
    payload = {
        "count": "true",
        "select": "title, chunk",  # 검색할 필드
        "vectorQueries": [{
            "vector": query_embedding(question),
            "fields": "text_vector",  # 벡터 필드명
            "kind": "vector",
            "exhaustive": "true",
            "k": top_k
        }]
    }
    response = requests.post(base_url, params=query_params, headers=headers, json=payload)
    response_body = response.json()
    return response_body

In [7]:
# 하이브리드 검색 + 시멘틱
def hybrid_semantic_search(question, index_name, top_p = 5, top_k = 5):
    base_url = f'{search_endpoint}/indexes/{index_name}/docs/search'
    query_params = {'api-version': '2023-11-01'}
    headers = {'Content-Type': 'application/json', 'api-key': search_key}
    payload = {
        "search": question,
        "select": "title, chunk",  # 검색할 필드
        "queryType": "semantic",
        "semanticConfiguration": f"{index_name}-semantic-configuration",  # 시멘틱 구성 이름
        "captions": "extractive",
        "answers": "extractive",
        "top": top_p,
        "vectorQueries": [{
            "vector": query_embedding(question),
            "fields": "text_vector",  # 벡터 필드명
            "kind": "vector",
            "exhaustive": "true",
            "k": top_k
        }]
    }
    response = requests.post(base_url, params=query_params, headers=headers, json=payload)
    response_body = response.json()
    return response_body

실행

In [8]:
query = "암보험"
index_name = "rag-test-index"

In [9]:
text_search(query, index_name)

{'@odata.context': "https://workshopsaisearch01.search.windows.net/indexes('rag-test-index')/$metadata#docs(*)",
 '@odata.count': 0,
 'value': []}

In [10]:
vector_search(query, index_name)

{'@odata.context': "https://workshopsaisearch01.search.windows.net/indexes('rag-test-index')/$metadata#docs(*)",
 '@odata.count': 5,
 'value': [{'@search.score': 0.8807286,
   'chunk': "보험Ⅱ(비갱신형)  \n\n무배당 \n\n암진단 10년 전기납 80.0% \n\n개별상품 위험보험료는 \n\n일반암 진단: 1,000만원(최초1회한) \n\n소액암 진단 : 600만원(최초1회한) \n\n소액질병 진단: 200만원(각각 최초1회한) \n\n지급 기준으로 산출 \n\ne특정 고액암 진단보장\n\n보험(비갱신형) \n\n무배당 \n\n암진단 10년 전기납 4.7% \n\n개별상품 위험보험료는 \n\n특정 고액암 진단 : 1,000만원(최초1회한) \n\n지급 기준으로 산출 \n\ne통합암보장보험[전이포\n\n함,일반암](비갱신형)  \n\n무배당 \n\n암진단 10년 전기납 128.8% \n\n개별상품 위험보험료는 \n\n일반암 진단: 1,000만원(최초1회한) \n\n소액암 진단 : 600만원(최초1회한) \n\n소액질병 진단: 200만원(각각 최초1회한) \n\n지급 기준으로 산출 \n\n※ 개별상품의 ‘일반암’은 ‘암(직∙결장암, 유방암, 여성생식기암, 전립선암, 기타피부암, 중증이외의 갑상\n\n선암, 대장점막내암 제외)’를 말합니다. \n\n※ 개별상품의 '소액암'은 직∙결장암, 유방암, 여성생식기암, 전립선암을 말합니다. \n\n※ 개별상품의 ‘소액질병’은 중증이외의 갑상선암, 기타피부암, 경계성종양, 제자리암, 대장점막내암을 \n\n말합니다. \n\n※ 개별상품의 '통합암'은 초기 이외의 유방ㆍ생식기암(전이포함), 두경부암(전이포함), 위암 및 \n\n식도암(전이포함), 소장ㆍ대장ㆍ항문암 및 기타암(전이포함), 간ㆍ〮낭ㆍ〮도암 및 췌장암(전이포함), \n\n흉곽내기관ㆍ중피성암 및 연조직암(전이포함), 골ㆍ피부

In [11]:
hybrid_semantic_search(query, index_name)

{'@odata.context': "https://workshopsaisearch01.search.windows.net/indexes('rag-test-index')/$metadata#docs(*)",
 '@search.answers': [{'key': '7099f6774d14_aHR0cHM6Ly93b3Jrc2hvcHNzYTAxLmJsb2IuY29yZS53aW5kb3dzLm5ldC9kYXRhLyVFRCU5NSU5QyVFRCU5OSU5NCVFQyU4MyU5RCVFQiVBQSU4NSUyMGUlRUMlOEIlOUMlRUElQjclQjglRUIlOEIlODglRUMlQjIlOTglRUMlOTUlOTQlRUIlQjMlQjQlRUQlOTclOTglMjAlRUIlQUMlQjQlRUIlQjAlQjAlRUIlOEIlQjlfJUVDJTgzJTgxJUVEJTkyJTg4JUVDJTlBJTk0JUVDJTk1JUJEJUVDJTg0JTlDXzIwMjUwNDAxLnBkZg2_pages_22',
   'text': '보험Ⅱ(비갱신형)    무배당   암진단 10년 전기납 80.0%   개별상품 위험보험료는   일반암 진단: 1,000만원(최초1회한)   소액암 진단 : 600만원(최초1회한)   소액질병 진단: 200만원(각각 최초1회한)   지급 기준으로 산출   e특정 고액암 진단보장  보험(비갱신형)   무배당   암진단 10년 전기납 4.7%   개별상품 위험보험료는   특정 고액암 진단 : 1,000만원(최초1회한)   지급 기준으로 산출   e통합암보장보험[전이포  함,일반암](비갱신형)    무배당   암진단 10년 전기납 128.8%   개별상품 위험보험료는   일반암 진단: 1,000만원(최초1회한)   소액암...',
   'highlights': '보험<em>Ⅱ(비갱신형)    무배당   암진단 10년 전기납 80.0% </em>  개별상품 위험보험료는   일반암 진단: 1,000만원(최초1회한)   소액암 진단 : 600만원(최초1회한)   소액질병 진단: 200만원(